In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv
import random
import time
import numpy as np

In [48]:
#create df
df = pd.read_csv('databaseUpdated-01.18.2024.csv')

df.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Magomed Ankalaev vs Johnny Walker,Magomed Ankalaev,Johnny Walker,Magomed Ankalaev,UFC Fight Night: Ankalaev vs. Walker 2,Marc Goddard,KO/TKO,01.13.2024,UFC Apex,no,...,0.0,0.0,2,2:42,"6' 3""","6' 6""","75""","82""",Orthodox,Orthodox
1,Jim Miller vs Gabriel Benitez,Jim Miller,Gabriel Benitez,Jim Miller,UFC Fight Night: Ankalaev vs. Walker 2,Dan Miragliotta,Submission,01.13.2024,UFC Apex,no,...,1.0,0.0,3,3:25,"5' 8""","5' 8""","71""","71""",Southpaw,Southpaw
2,Mario Bautista vs Ricky Simon,Ricky Simon,Mario Bautista,Mario Bautista,UFC Fight Night: Ankalaev vs. Walker 2,Mark Smith,Decision - Unanimous,01.13.2024,UFC Apex,no,...,0.0,0.0,3,5:00,"5' 6""","5' 9""","69""","69""",Orthodox,Switch
3,Brunno Ferreira vs Phil Hawes,Phil Hawes,Brunno Ferreira,Brunno Ferreira,UFC Fight Night: Ankalaev vs. Walker 2,Chris Tognoni,KO/TKO,01.13.2024,UFC Apex,no,...,0.0,0.0,1,4:55,"6' 0""","5' 10""","77""","72""",Orthodox,Orthodox
4,Waldo Cortes Acosta vs Andrei Arlovski,Andrei Arlovski,Waldo Cortes Acosta,Waldo Cortes Acosta,UFC Fight Night: Ankalaev vs. Walker 2,Marc Goddard,Decision - Unanimous,01.13.2024,UFC Apex,no,...,0.0,0.0,3,5:00,"6' 3""","6' 4""","77""","78""",Orthodox,Orthodox


In [49]:
#generate proxyList
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 39


In [50]:
def getProxyUserAgent():
    for i in proxylist:
        #test to see if website is accessible
        userAgents = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36','Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36','Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0','Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/116.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188','insomnia/8.4.5','Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 6a) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 6 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 13; Pixel 7 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 12; moto g pure) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (Linux; Android 12; moto g stylus 5G) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36v','Mozilla/5.0 (Linux; Android 13; SM-G998U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36','Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/13.2b11866 Mobile/16A366 Safari/605.1.15','Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1']
        userAgent = userAgents[random.randint(0,len(userAgents)-1)]
        try:
            payload = ''
            headers = {
                "cookie": "_tapology_mma_session=zo%252F9cU1na2qfRAvGf4E%252FBdxyMqNEgZbmqUYGnDxTYw%252BkgkuquO4qPimSMq%252FNc4fAxpLoIGwkl%252Fvw%252FhO04rqrL1PuS7516fTWktFyWhkz6YUy7MvWUVyjNQ7R26QYA8TeQruG5w%252B6RAj71bMME5MxoLjpSOs%252FyinaA6qsprmBZ2LnagrpzZ7bxaPvk7o%252FohTgZgxpo0FGWGaDdHERD%252B3Bt3C3ucykGOC7WB65EM8xB6C8gMNzGsvEu8FbvGnbMaoqzGzx%252FjRprzFrLN4mE5vdrJ1fsjoDV9cn5NEE2zI%253D--IkSvjCRiM5qMGaKi--o55iUwoIEhI6T8jM4UW6tA%253D%253D",
                "User-Agent": f'{userAgent}'
            }

            url = "https://www.tapology.com"
        
            #site request
            #soup = BeautifulSoup(driver.page_source, 'lxml')
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        except:
            pass
        #check to see if uesr agent was the issue
        try:
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            #soup = BeautifulSoup(driver.page_source, 'lxml')
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        #if user agent is not new issue, wait for next IP
        except:
            pass
        try:
            print("Waiting for new IP...")
            ipurl = "https://ipecho.net/plain"
            ipHeader = {"User-Agent": "insomnia/8.4.5"}
            ipr = requests.request("GET", url=ipurl, headers=ipHeader)
            soup = BeautifulSoup(ipr.content, 'html.parser')
            currentIp = soup.text.strip()
            newIP = soup.text.strip()
            while(currentIp == newIP):
                ipurl = "https://ipecho.net/plain"
                ipHeader = {"User-Agent": "insomnia/8.4.5"}
                ipr = requests.request("GET", url=ipurl, headers=ipHeader)
                soup = BeautifulSoup(ipr.content, 'html.parser')
                newIP = soup.text.strip()
                time.sleep(15)
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            #soup = BeautifulSoup(driver.page_source, 'lxml')
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader
        except:
            print("Maintenence required...")
            input("Press enter to continue")
            userAgents.remove(userAgent)
            userAgent = userAgents[random.randint(0,len(userAgents)-1)]
            payload = ''
            headers = {'User-Agent':f'{userAgent}',
            "cookie": "_tapology_mma_session=TCLB17ieOPnBLmCBTuxpX8s3uBODZMN3jL3jBbFhwPoywfbzG7gyvp%252BAzbOk4gOZ%252FOCykOUwcpEoJBwoj2rJyiMxdHWSaiLFkBYjfuUDpZ2VY6ECFn6rpTPmUBY1Zr2anIqiklY6fz9yQlBkPAhcx%252BWSzVgsc%252B%252F8UCqkb6WnM6xr8GUikb8U2UkMVYZ3Nj1dIA0vbXpDhKykqgW%252BCnlyglp8rtdlQ37m0SaYWjLDthG7Tik3idUvGlSXFAU55zAnxz6UNncMNhTbo5ltINfso54j60i7hOq0utNOz9w%253D--ChZexYwNpevIJ%252BV8--HhKStLELFfNWYOTZIAKM6Q%253D%253D"}
            url = "https://www.tapology.com/fightcenter/bouts/2974-ufc-64-clay-the-carpenter-guida-vs-justin-pretty-boy-james"
        
            #site request
            #soup = BeautifulSoup(driver.page_source, 'lxml')
            response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{i}"})
            soup = BeautifulSoup(response.content, 'html.parser')
            boutInfo = soup.find('div', class_=re.compile('right'))
            labels = boutInfo.find_all('li')
            proxyheader = [i,userAgent]
            return proxyheader


In [51]:
df.replace('---', np.nan, inplace=True)
df.replace('--', np.nan, inplace=True)

In [54]:
#find nans to fix
columns_of_interest = ['redCorner_reach', 'blueCorner_reach', 'redCorner_stance', 'blueCorner_stance', 'redCorner_age', 'blueCorner_age',]

first_row_with_nan = df[df[columns_of_interest].isnull().any(axis=1)].head(1)

values_to_print = first_row_with_nan[['redCorner', 'blueCorner', 'redCorner_height', 'blueCorner_height', 'redCorner_reach', 'blueCorner_reach', 'redCorner_stance', 'blueCorner_stance', 'redCorner_age', 'blueCorner_age', 'date']]

total_nans = df[columns_of_interest].isnull().sum().sum()

print(total_nans)
print(values_to_print)

1582
           redCorner      blueCorner redCorner_height blueCorner_height  \
2581  Maurice Greene  Michel Batista            6' 7"             6' 3"   

     redCorner_reach blueCorner_reach redCorner_stance blueCorner_stance  \
2581             80"              NaN         Orthodox          Orthodox   

      redCorner_age  blueCorner_age        date  
2581           34.0            32.0  11.30.2018  


In [53]:
#6161
for index, row in df.loc[:2415].iterrows():
    reach = None
    if row['blueCorner_reach'] is np.nan or row['blueCorner_reach'] is np.nan:
        fighter = row['blueCorner']
        fighterList = fighter.split(' ')
        if len(fighterList) == 1:
            url = f'https://www.tapology.com/search?term={fighterList[0]}&commit=Submit&model%5Bfighters%5D=fightersSearch'

            #getProxyHeader
            proxyheader = getProxyUserAgent()
            proxy = proxyheader[0]
            userAgent = proxyheader[1]

            #define headers
            headers = {
            "cookie": "_tapology_mma_session=nUbYZsmfBxCMml3EfKnycLX59tOCxmooAxj8ifN56kFksGAYkSXxKRVIy7bT2%252FXP9kIGTPOV3O%252BN1%252B4GWiNAjhG9S8sCeUtN3W2bwuxKzi8XGqMMYBSU1NWLZnSqZSIME2kzEx4xwAIBlWjakwZAGvZYuo4mIAV1OmuP%252B6n%252FU5ps1xWx1z%252BHrPIex61hMrNZpQfSwZNsbHcOLcrEjFLxfWUeIq7knZyMBQPGrlD%252BcSOCROXMSCXkEe7DbSgmZQLI5PVfbsBzN29g6irLU4XHrtsn%252FeZ%252BWFBGkAz1fEo%253D--thae17JwlRQtS%252BZB--4vVdpKGWVPPNtOCCVdkaWg%253D%253D",
            "User-Agent": f'{userAgent}'
            }

            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')

            #get href
            linkPart = soup.find('td', class_=re.compile('altA'))
            a = linkPart.find('a')
            href = a['href']

            url = f'https://tapology.com{href}'
            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')
            
            div = soup.find('div', class_=re.compile('details details_two_columns'))
            lis = div.find_all('li')
            for li in lis:
                if 'Reach:' in li.text.strip():
                    spans = li.find_all('span')
                    if len(spans)>1:
                        reachTemp = spans[1].text.strip()
                        reachList = reachTemp.split(' ')
                        reach = str(reachTemp[0])
                        df.at[index, 'blueCorner_reach'] = reach
        if len(fighterList) == 2:
            url = f'https://www.tapology.com/search?term={fighterList[0]}+{fighterList[1]}&commit=Submit&model%5Bfighters%5D=fightersSearch'
            
            #getProxyHeader
            proxyheader = getProxyUserAgent()
            proxy = proxyheader[0]
            userAgent = proxyheader[1]

            #define headers
            headers = {
            "cookie": "_tapology_mma_session=nUbYZsmfBxCMml3EfKnycLX59tOCxmooAxj8ifN56kFksGAYkSXxKRVIy7bT2%252FXP9kIGTPOV3O%252BN1%252B4GWiNAjhG9S8sCeUtN3W2bwuxKzi8XGqMMYBSU1NWLZnSqZSIME2kzEx4xwAIBlWjakwZAGvZYuo4mIAV1OmuP%252B6n%252FU5ps1xWx1z%252BHrPIex61hMrNZpQfSwZNsbHcOLcrEjFLxfWUeIq7knZyMBQPGrlD%252BcSOCROXMSCXkEe7DbSgmZQLI5PVfbsBzN29g6irLU4XHrtsn%252FeZ%252BWFBGkAz1fEo%253D--thae17JwlRQtS%252BZB--4vVdpKGWVPPNtOCCVdkaWg%253D%253D",
            "User-Agent": f'{userAgent}'
            }

            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')

            #get href
            linkPart = soup.find('td', class_=re.compile('altA'))
            a = linkPart.find('a')
            href = a['href']

            url = f'https://tapology.com{href}'
            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')
            
            div = soup.find('div', class_=re.compile('details details_two_columns'))
            lis = div.find_all('li')
            for li in lis:
                if 'Reach:' in li.text.strip():
                    spans = li.find_all('span')
                    if len(spans)>1:
                        reachTemp = spans[1].text.strip()
                        reachList = reachTemp.split(' ')
                        reach = str(reachTemp[0])
                        df.at[index, 'blueCorner_reach'] = reach
        if len(fighterList) == 3:
            url =f'https://www.tapology.com/search?term={fighterList[0]}+{fighterList[1]}+{fighterList[2]}&commit=Submit&model%5Bfighters%5D=fightersSearch'
            #getProxyHeader
            proxyheader = getProxyUserAgent()
            proxy = proxyheader[0]
            userAgent = proxyheader[1]

            #define headers
            headers = {
            "cookie": "_tapology_mma_session=nUbYZsmfBxCMml3EfKnycLX59tOCxmooAxj8ifN56kFksGAYkSXxKRVIy7bT2%252FXP9kIGTPOV3O%252BN1%252B4GWiNAjhG9S8sCeUtN3W2bwuxKzi8XGqMMYBSU1NWLZnSqZSIME2kzEx4xwAIBlWjakwZAGvZYuo4mIAV1OmuP%252B6n%252FU5ps1xWx1z%252BHrPIex61hMrNZpQfSwZNsbHcOLcrEjFLxfWUeIq7knZyMBQPGrlD%252BcSOCROXMSCXkEe7DbSgmZQLI5PVfbsBzN29g6irLU4XHrtsn%252FeZ%252BWFBGkAz1fEo%253D--thae17JwlRQtS%252BZB--4vVdpKGWVPPNtOCCVdkaWg%253D%253D",
            "User-Agent": f'{userAgent}'
            }

            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')

            #get href
            linkPart = soup.find('td', class_=re.compile('altA'))
            a = linkPart.find('a')
            href = a['href']

            url = f'https://tapology.com{href}'
            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')
            
            div = soup.find('div', class_=re.compile('details details_two_columns'))
            lis = div.find_all('li')
            for li in lis:
                if 'Reach:' in li.text.strip():
                    spans = li.find_all('span')
                    if len(spans)>1:
                        reachTemp = spans[1].text.strip()
                        reachList = reachTemp.split(' ')
                        reach = str(reachTemp[0])
                        df.at[index, 'blueCorner_reach'] = reach
        if len(fighterList) == 4:
            url = f'https://www.tapology.com/search?term={fighterList[0]}+{fighterList[1]}+{fighterList[2]}+{fighterList[3]}&commit=Submit&model%5Bfighters%5D=fightersSearch'
            #getProxyHeader
            proxyheader = getProxyUserAgent()
            proxy = proxyheader[0]
            userAgent = proxyheader[1]

            #define headers
            headers = {
            "cookie": "_tapology_mma_session=nUbYZsmfBxCMml3EfKnycLX59tOCxmooAxj8ifN56kFksGAYkSXxKRVIy7bT2%252FXP9kIGTPOV3O%252BN1%252B4GWiNAjhG9S8sCeUtN3W2bwuxKzi8XGqMMYBSU1NWLZnSqZSIME2kzEx4xwAIBlWjakwZAGvZYuo4mIAV1OmuP%252B6n%252FU5ps1xWx1z%252BHrPIex61hMrNZpQfSwZNsbHcOLcrEjFLxfWUeIq7knZyMBQPGrlD%252BcSOCROXMSCXkEe7DbSgmZQLI5PVfbsBzN29g6irLU4XHrtsn%252FeZ%252BWFBGkAz1fEo%253D--thae17JwlRQtS%252BZB--4vVdpKGWVPPNtOCCVdkaWg%253D%253D",
            "User-Agent": f'{userAgent}'
            }

            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')

            #get href
            linkPart = soup.find('td', class_=re.compile('altA'))
            a = linkPart.find('a')
            href = a['href']

            url = f'https://tapology.com{href}'
            #site request
            site = requests.get(url, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
            soup = BeautifulSoup(site.content, 'html.parser')
            
            div = soup.find('div', class_=re.compile('details details_two_columns'))
            lis = div.find_all('li')
            for li in lis:
                if 'Reach:' in li.text.strip():
                    spans = li.find_all('span')
                    if len(spans)>1:
                        reachTemp = spans[1].text.strip()
                        reachList = reachTemp.split(' ')
                        reach = str(reachTemp[0])
                        df.at[index, 'blueCorner_reach'] = reach
        